# Language Modeling with RNN

## Language Modeling predicts the next word in a sequence, which is crucial for tasks like text generation.

### Steps:

- Data Preparation: Prepare a text corpus (e.g., Shakespeare's works).
- Preprocessing: Create sequences of text, tokenize, and prepare input-output pairs.
- Model: Build an RNN model to predict the next word in a sequence.
- Training: Train the model on the text corpus.
- Generation: Generate new text by sampling from the model.

In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [25]:
# Example text data (replace with your text corpus)
text = "Your large text corpus here. It can be multiple sentences."

In [26]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [27]:
# Prepare input sequences
input_sequences = []
for line in text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [28]:
# Pad sequences and create predictors and labels
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [29]:
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [20]:
# Reshape input to be 3D (batch_size, timesteps, input_dim)
X = np.expand_dims(X, axis=-1)

In [30]:
# Build RNN model
model = Sequential([
    Embedding(total_words, 50, input_length=max_sequence_len-1),
    LSTM(100),
    Dense(total_words, activation='softmax')
])

In [32]:
# Model compilation
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
# Train the model
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
1/1 - 2s - 2s/step - accuracy: 0.0000e+00 - loss: 2.3991
Epoch 2/100
1/1 - 0s - 17ms/step - accuracy: 0.2500 - loss: 2.3929
Epoch 3/100
1/1 - 0s - 20ms/step - accuracy: 0.3750 - loss: 2.3867
Epoch 4/100
1/1 - 0s - 26ms/step - accuracy: 0.3750 - loss: 2.3805
Epoch 5/100
1/1 - 0s - 17ms/step - accuracy: 0.3750 - loss: 2.3742
Epoch 6/100
1/1 - 0s - 17ms/step - accuracy: 0.3750 - loss: 2.3677
Epoch 7/100
1/1 - 0s - 16ms/step - accuracy: 0.3750 - loss: 2.3610
Epoch 8/100
1/1 - 0s - 17ms/step - accuracy: 0.3750 - loss: 2.3539
Epoch 9/100
1/1 - 0s - 18ms/step - accuracy: 0.3750 - loss: 2.3466
Epoch 10/100
1/1 - 0s - 17ms/step - accuracy: 0.3750 - loss: 2.3387
Epoch 11/100
1/1 - 0s - 16ms/step - accuracy: 0.3750 - loss: 2.3304
Epoch 12/100
1/1 - 0s - 17ms/step - accuracy: 0.3750 - loss: 2.3214
Epoch 13/100
1/1 - 0s - 17ms/step - accuracy: 0.3750 - loss: 2.3118
Epoch 14/100
1/1 - 0s - 18ms/step - accuracy: 0.3750 - loss: 2.3014
Epoch 15/100
1/1 - 0s - 17ms/step - accuracy: 0.3750 - 

In [34]:
# Text generation function
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word